In [ ]:
!pip -q install "pillow<12"
!pip -q install -U transformers datasets evaluate accelerate opencv-python scipy scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.4 MB/s eta 0:00:00


In [ ]:
# تثبيت SAM (مرة واحدة)
!pip -q install git+https://github.com/facebookresearch/segment-anything.git
!pip -q install -U pycocotools opencv-python matplotlib

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import urllib
import os, glob, numpy as np
import urllib.request
from PIL import Image
import random
import cv2
from scipy.io import loadmat
import glob, random
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from transformers import TrainingArguments, Trainer
import evaluate
import torch
from segment_anything import sam_model_registry, SamPredictor
import numpy as np
from PIL import Image
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
DRIVE_ROOT = "/content/drive/MyDrive/NYU_Depth_Dataset"
DRIVE_ROOT1 = "/content/drive/MyDrive/NYU_Depth_Dataset/output"
IMG_DIR   = os.path.join(DRIVE_ROOT1, "images")   #  images
LABEL_DIR = os.path.join(DRIVE_ROOT1, "labels")   #  labels
MAT_PATH  = os.path.join(DRIVE_ROOT, "nyu_depth_v2_labeled.mat")

assert os.path.isdir(IMG_DIR), IMG_DIR
assert os.path.isdir(LABEL_DIR), LABEL_DIR
assert os.path.isfile(MAT_PATH), f"ملف .mat غير موجود: {MAT_PATH}"

print("IMG_DIR:", IMG_DIR)
print("LABEL_DIR:", LABEL_DIR)
print("MAT_PATH:", MAT_PATH)

IMG_DIR: /content/drive/MyDrive/NYU_Depth_Dataset/output/images
LABEL_DIR: /content/drive/MyDrive/NYU_Depth_Dataset/output/labels
MAT_PATH: /content/drive/MyDrive/NYU_Depth_Dataset/nyu_depth_v2_labeled.mat


In [ ]:
#فحص سريع للـ labels
def load_id_mask(path):
    m = np.array(Image.open(path))
    if m.ndim == 3:
        m = m[...,0]
    return m

sample = random.choice(glob.glob(os.path.join(LABEL_DIR, "*.png")))
m = load_id_mask(sample)
print("Sample:", os.path.basename(sample))
print("Shape:", m.shape)
print("Unique IDs (first 50):", np.unique(m)[:50])
print("Max ID:", int(m.max()))

Sample: label_1001.png
Shape: (480, 640)
Unique IDs (first 50): [  0   5  11  21  28  64  84  85  88  89  98  99 119 137 141 157 177 255]
Max ID: 255


In [ ]:
import h5py

f = h5py.File(MAT_PATH, "r")

print("Top/H5 keys:", list(f.keys()), "... total:", len(f.keys()))

# ---- 1) قراءة أسماء الفئات من names (References) ----
assert "names" in f, "ما لقيت المفتاح 'names' داخل الملف."

names_refs = np.array(f["names"]).squeeze()  # غالباً array من object references

def ref_to_string(h5file, ref):
    """
    يفك HDF5 object reference ويحوّله لنص.
    كثير ملفات MATLAB v7.3 تخزن النص كسلسلة char codes (uint16) أو bytes.
    """
    obj = h5file[ref]              # Dataset أو Group مشار إليه
    arr = np.array(obj)

    # 1) إذا كان bytes مباشرة
    if arr.dtype.kind in ("S",):   # byte strings
        s = b"".join(arr.flatten()).decode("utf-8", errors="ignore")
        return s.strip("\x00").strip()

    # 2) MATLAB chars غالباً uint16 (Unicode code units)
    if arr.dtype == np.uint16 or arr.dtype == np.uint8 or arr.dtype == np.int16:
        # MATLAB يخزن char arrays بشكل عمودي أحياناً -> جرّب order='F'
        flat = arr.flatten(order="F")
        # حوّل لأحرف
        s = "".join(chr(int(c)) for c in flat if int(c) != 0)
        return s.strip()

    # 3) fallback
    return str(arr.squeeze())

# بعض الملفات يكون names_refs على شكل 2D (1,N) أو (N,1)
names_refs_flat = names_refs.flatten()

class_names = [ref_to_string(f, r) for r in names_refs_flat]
print("Example class names:", class_names[:30])

# ---- 2) إيجاد IDs عبر كلمات مفتاحية ----
def find_ids_by_keywords(names, keywords):
    ids = []
    for i, n in enumerate(names):
        ln = (n or "").lower()
        if any(kw in ln for kw in keywords):
            ids.append(i)
    return ids

wall_ids_raw    = find_ids_by_keywords(class_names, ["wall"])
floor_ids_raw   = find_ids_by_keywords(class_names, ["floor"])
ceiling_ids_raw = find_ids_by_keywords(class_names, ["ceiling", "ceil"])

print("Raw matches counts:", {
    "wall": len(wall_ids_raw),
    "floor": len(floor_ids_raw),
    "ceiling": len(ceiling_ids_raw),
})
print("Some wall matches:", [class_names[i] for i in wall_ids_raw[:10]])
print("Some floor matches:", [class_names[i] for i in floor_ids_raw[:10]])
print("Some ceiling matches:", [class_names[i] for i in ceiling_ids_raw[:10]])

assert len(wall_ids_raw) > 0 and len(floor_ids_raw) > 0 and len(ceiling_ids_raw) > 0, \
    "لسّا ما لقيت wall/floor/ceiling. ابعتيلي أول 80 اسم من class_names لنضبط كلمات البحث."

# ---- 3) كشف 0-based / 1-based من PNG labels ----
png_paths = glob.glob(os.path.join(LABEL_DIR, "*.png"))
assert len(png_paths) > 0, f"ما لقيت PNG داخل {LABEL_DIR}"

png_sample = load_id_mask(random.choice(png_paths))
uniq = set(np.unique(png_sample).tolist())

def pick_base(raw_ids, uniq_set):
    if any(i in uniq_set for i in raw_ids):
        return 0
    if any((i + 1) in uniq_set for i in raw_ids):
        return 1
    return 0

BASE = max(
    pick_base(wall_ids_raw, uniq),
    pick_base(floor_ids_raw, uniq),
    pick_base(ceiling_ids_raw, uniq),
)

print("Detected label base (0=0-based, 1=1-based):", BASE)

WALL_IDS    = [i + BASE for i in wall_ids_raw]
FLOOR_IDS   = [i + BASE for i in floor_ids_raw]
CEILING_IDS = [i + BASE for i in ceiling_ids_raw]

print("Candidate WALL IDs:", WALL_IDS[:15])
print("Candidate FLOOR IDs:", FLOOR_IDS[:15])
print("Candidate CEILING IDs:", CEILING_IDS[:15])




Top/H5 keys: ['#refs#', '#subsystem#', 'accelData', 'depths', 'images', 'instances', 'labels', 'names', 'namesToIds', 'rawDepthFilenames', 'rawDepths', 'rawRgbFilenames', 'sceneTypes', 'scenes'] ... total: 14
Example class names: ['book', 'bottle', 'cabinet', 'ceiling', 'chair', 'cone', 'counter', 'dishwasher', 'faucet', 'fire extinguisher', 'floor', 'garbage bin', 'microwave', 'paper towel dispenser', 'paper', 'pot', 'refridgerator', 'stove burner', 'table', 'unknown', 'wall', 'bowl', 'magnet', 'sink', 'air vent', 'box', 'door knob', 'door', 'scissor', 'tape dispenser']
Raw matches counts: {'wall': 6, 'floor': 3, 'ceiling': 1}
Some wall matches: ['wall', 'wall decoration', 'wall stand', 'wall hand sanitizer dispenser', 'wallet', 'wall divider']
Some floor matches: ['floor', 'floor mat', 'floor trim']
Some ceiling matches: ['ceiling']
Detected label base (0=0-based, 1=1-based): 1
Candidate WALL IDs: [21, 186, 295, 440, 661, 800]
Candidate FLOOR IDs: [11, 143, 868]
Candidate CEILING IDs

In [ ]:
import re

def find_ids_regex(names, pattern):
    rgx = re.compile(pattern, flags=re.IGNORECASE)
    return [i for i, n in enumerate(names) if rgx.search(n)]

# floor: نريد "floor" لوحدها أو يبدأ فيها
floor_ids_raw = find_ids_regex(class_names, r"^floor$|^floor\s")

# ceiling: غالبًا "ceiling" لوحدها
ceiling_ids_raw = find_ids_regex(class_names, r"^ceiling$")

# wall: نريد "wall" لوحدها أو يبدأ فيها "wall ..."
# (هذا يستبعد wallet لأنه يبدأ بـ wal**let** وليس wall + مسافة)
wall_ids_raw = find_ids_regex(class_names, r"^wall$|^wall\s")

print("Filtered matches:", {
    "wall": (len(wall_ids_raw), [class_names[i] for i in wall_ids_raw]),
    "floor": (len(floor_ids_raw), [class_names[i] for i in floor_ids_raw]),
    "ceiling": (len(ceiling_ids_raw), [class_names[i] for i in ceiling_ids_raw]),
})


Filtered matches: {'wall': (5, ['wall', 'wall decoration', 'wall stand', 'wall hand sanitizer dispenser', 'wall divider']), 'floor': (3, ['floor', 'floor mat', 'floor trim']), 'ceiling': (1, ['ceiling'])}


In [ ]:
BASE = 1

WALL_IDS    = [i + BASE for i in wall_ids_raw]
FLOOR_IDS   = [i + BASE for i in floor_ids_raw]
CEILING_IDS = [i + BASE for i in ceiling_ids_raw]

print("FINAL WALL IDS:", WALL_IDS)
print("FINAL FLOOR IDS:", FLOOR_IDS)
print("FINAL CEILING IDS:", CEILING_IDS)


FINAL WALL IDS: [21, 186, 295, 440, 800]
FINAL FLOOR IDS: [11, 143, 868]
FINAL CEILING IDS: [4]


In [ ]:
#اختيار ID “الأقوى” لكل فئة حسب تكراره + تحويل كل الـ labels إلى 0/1/2/3
label_files = sorted(glob.glob(os.path.join(LABEL_DIR, "*.png")))
assert len(label_files) > 0

cnt = Counter()
for p in label_files[:200]:  # كفاية 200 للتقدير (سريع)
    src = load_id_mask(p)
    vals, counts = np.unique(src, return_counts=True)
    for v, c in zip(vals, counts):
        cnt[int(v)] += int(c)

def best_id(candidates):
    candidates = [c for c in candidates if c in cnt]
    assert len(candidates) > 0, "ولا ID من المرشحين ظهر بالليبلات!"
    return max(candidates, key=lambda x: cnt[x])

BEST_WALL    = best_id(WALL_IDS)
BEST_FLOOR   = best_id(FLOOR_IDS)
BEST_CEILING = best_id(CEILING_IDS)

print("BEST_WALL:", BEST_WALL, "BEST_FLOOR:", BEST_FLOOR, "BEST_CEILING:", BEST_CEILING)

OUT_MASK_DIR = os.path.join(DRIVE_ROOT1, "labels_0_1_2_3")
os.makedirs(OUT_MASK_DIR, exist_ok=True)

for p in label_files:
    src = load_id_mask(p)
    out = np.zeros_like(src, dtype=np.uint8)  # background=0
    out[src == BEST_WALL]    = 1
    out[src == BEST_FLOOR]   = 2
    out[src == BEST_CEILING] = 3
    Image.fromarray(out, mode="L").save(os.path.join(OUT_MASK_DIR, os.path.basename(p)))

chk = load_id_mask(os.path.join(OUT_MASK_DIR, os.path.basename(label_files[0])))
print("Converted mask unique:", np.unique(chk))
print("Saved masks to:", OUT_MASK_DIR)

BEST_WALL: 21 BEST_FLOOR: 11 BEST_CEILING: 4


/tmp/ipython-input-817774387.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(out, mode="L").save(os.path.join(OUT_MASK_DIR, os.path.basename(p)))


Converted mask unique: [0 1 2 3]
Saved masks to: /content/drive/MyDrive/NYU_Depth_Dataset/output/labels_0_1_2_3


In [ ]:
# 1) دالة تستخرج رقم/ID من اسم الملف (أول رقم بتلاقيه)
def extract_id(filename: str):
    m = re.search(r"\d+", filename)
    return m.group(0) if m else None

img_files = sorted([f for f in os.listdir(IMG_DIR)
                    if f.lower().endswith((".jpg", ".jpeg", ".png"))])

msk_files = sorted([f for f in os.listdir(OUT_MASK_DIR)
                    if f.lower().endswith(".png")])

mask_map = {}
for m in msk_files:
    mid = extract_id(m)
    if mid:
        mask_map[mid] = os.path.join(OUT_MASK_DIR, m)

pairs = []
missing = 0
for im in img_files:
    iid = extract_id(im)
    if iid and iid in mask_map:
        pairs.append((os.path.join(IMG_DIR, im), mask_map[iid]))
    else:
        missing += 1

print("Images:", len(img_files), "Masks:", len(msk_files))
print("Matched pairs:", len(pairs), "Missing images without mask:", missing)

assert len(pairs) > 50, "المطابقات قليلة جدًا—راجعي أسماء الصور/الماسكات واستخراج الـ id."

train_pairs, val_pairs = train_test_split(pairs, test_size=0.15, random_state=42)
print("Train:", len(train_pairs), "Val:", len(val_pairs))

Images: 1449 Masks: 1449
Matched pairs: 1449 Missing images without mask: 0
Train: 1231 Val: 218


In [ ]:
# تدريب SegFormer بشكل قوي (Augment + fp16 + cosine scheduler) + حفظ على Drive
id2label = {0:"background", 1:"wall", 2:"floor", 3:"ceiling"}
label2id = {v:k for k,v in id2label.items()}

CKPT = "nvidia/segformer-b2-finetuned-ade-512-512"

processor = SegformerImageProcessor.from_pretrained(CKPT)
model = SegformerForSemanticSegmentation.from_pretrained(
    CKPT,
    num_labels=4,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

# Augment بسيط لكنه فعال بدون ما يخرّب الواقعية
def random_hflip(image, mask, p=0.5):
    if np.random.rand() < p:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
        mask  = mask.transpose(Image.FLIP_LEFT_RIGHT)
    return image, mask

def random_color_jitter(image, p=0.5):
    if np.random.rand() < p:
        import torchvision.transforms as T
        jitter = T.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.10, hue=0.02)
        image = jitter(image)
    return image

class RoomStructDataset(Dataset):
    def __init__(self, pairs, train=True):
        self.pairs = pairs
        self.train = train

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, msk_path = self.pairs[idx]
        image = Image.open(img_path).convert("RGB")
        mask  = Image.open(msk_path)  # 0..3

        if self.train:
            image, mask = random_hflip(image, mask, p=0.5)
            image = random_color_jitter(image, p=0.5)

        enc = processor(image, mask, return_tensors="pt")
        return {k:v.squeeze(0) for k,v in enc.items()}

train_ds = RoomStructDataset(train_pairs, train=True)
val_ds   = RoomStructDataset(val_pairs, train=False)

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.from_numpy(logits)
    up = torch.nn.functional.interpolate(
        logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
    )
    pred = up.argmax(dim=1).cpu().numpy()
    out = metric.compute(
        predictions=pred,
        references=labels,
        num_labels=4,
        ignore_index=-1,
        reduce_labels=False,
    )
    return {"mean_iou": out["mean_iou"], "mean_accuracy": out["mean_accuracy"]}

SAVE_DIR = "/content/drive/MyDrive/segformer_room_struct"

args = TrainingArguments(
    output_dir=SAVE_DIR,
    learning_rate=5e-5,
    num_train_epochs=25,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,

    eval_strategy="epoch",
    save_strategy="epoch",

    lr_scheduler_type="cosine",
    warmup_ratio=0.06,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="mean_iou",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

trainer.train()
print(trainer.evaluate())

trainer.save_model(os.path.join(SAVE_DIR, "final"))
processor.save_pretrained(os.path.join(SAVE_DIR, "final"))
print("Saved final model to:", os.path.join(SAVE_DIR, "final"))


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([4, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy
1,No log,0.383223,0.717186,0.782658
2,0.694500,0.245850,0.784747,0.888320
3,0.694500,0.207123,0.810957,0.905783
4,0.245800,0.209224,0.809003,0.907206
5,0.180800,0.199084,0.812526,0.909700
6,0.180800,0.221859,0.811621,0.901935
7,0.153600,0.209925,0.818627,0.912482
8,0.153600,0.217116,0.819809,0.907179
9,0.135000,0.228956,0.818382,0.910023
10,0.120100,0.226101,0.812077,0.909781


/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible 

Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy
1,No log,0.383223,0.717186,0.782658
2,0.694500,0.245850,0.784747,0.888320
3,0.694500,0.207123,0.810957,0.905783
4,0.245800,0.209224,0.809003,0.907206
5,0.180800,0.199084,0.812526,0.909700
6,0.180800,0.221859,0.811621,0.901935
7,0.153600,0.209925,0.818627,0.912482
8,0.153600,0.217116,0.819809,0.907179
9,0.135000,0.228956,0.818382,0.910023
10,0.120100,0.226101,0.812077,0.909781


/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible 

/usr/local/lib/python3.12/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


{'eval_loss': 0.25056225061416626, 'eval_mean_iou': 0.8268510781330647, 'eval_mean_accuracy': 0.9082203592141851, 'eval_runtime': 16.7283, 'eval_samples_per_second': 13.032, 'eval_steps_per_second': 6.516, 'epoch': 25.0}
Saved final model to: /content/drive/MyDrive/segformer_room_struct/final
